In [50]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
# En este notebook representare las montañas más altas del mundo, rios rios mñas largos del mundo y de españa y, por último, el cámino de Santiago desde Roma

In [52]:
# En primer lugar habrá que instalar e importar las liberiras necesarias

In [53]:
pip install geopandas

In [54]:
pip install gpxpy 

In [55]:
import geopandas as gpd
import folium
import pandas as pd
from folium.plugins import MarkerCluster
import gpxpy
import os

In [56]:
#Cargo el shp de todos los países del mundo
#https://www.efrainmaps.es/descargas-gratuitas/mundo/

In [ ]:
df_mundo = gpd.read_file('/content/drive/MyDrive/COLAB/BDA/Paises_Mundo/Paises_Mundo.shp')
df_mundo.head(5)#muestro los 5 primeros paises

In [ ]:
df_mundo.plot(figsize=(15,30))

In [59]:
#Cargo el de los ríos del mundo
#https://www.efrainmaps.es/descargas-gratuitas/mundo/

In [ ]:
df_rios = gpd.read_file('/content/drive/MyDrive/COLAB/BDA/Hidrografia_Mundo/Hidrografia_Mundo.shp')
df_rios.head(5)

In [61]:
#Asigno un color diferente para los países y los ríos

In [62]:
style1 = {'fillColor': 'grey', 'color': 'grey'}#paises
style2 = {'fillColor': '#00FFFFFF', 'color': '#00FFFFFF'}#rios

In [ ]:
m = folium.Map(location=[43, -8.20],zoom_start=2)

folium.GeoJson(df_mundo,name="paises", style_function=lambda x:style1).add_to(m)#cargo los shp
folium.GeoJson(df_rios,name="rios", style_function=lambda x:style2).add_to(m)
folium.LayerControl().add_to(m)
m

In [64]:
#Cargo el de las montañas más grandes
#https://www.kaggle.com/datasets/keagle/mountains-dataset-with-coordinates-and-countries

In [65]:
df_montañas = gpd.read_file('/content/drive/MyDrive/COLAB/BDA/archive/Top montains data.csv')

In [ ]:
df_montañas.head(5)

In [67]:
#Guardo sus nombres

In [68]:
montes=df_montañas["Parent mountain"].unique()

In [69]:
#Lo convierto en un dataframe

In [70]:
df_montes = pd.DataFrame(montes, columns = ['montañas'])

In [71]:
#Corrijo algunos errores del dataframe

In [72]:
df_montes=df_montes.drop([0,41,45,37],axis=0)
df_montes=df_montes.reset_index()
df_montes=df_montes.replace({"Khunyang Chhish": "Kunyang Chhish"})
df_montes=df_montes.replace({"Labuche Himilaya": "Lobuche"})
df_montes=df_montes.drop(["index"],axis=1)

In [73]:
#Mediante la API de Nominatium consigo los coordenadas de esas montañas y las guardo en el dataframe

In [74]:
import requests
datos_montes=[]
for n in range (1,47):
  busqueda = df_montes["montañas"][n]

  search_url = 'https://nominatim.openstreetmap.org/search'
  payload = {'q': busqueda ,'format':'json'}

  search_response = requests.get(search_url,params=payload)

  latitude = search_response.json()[0]['lat']
  longitude = search_response.json()[0]['lon']

  datos_montes.append([df_montes["montañas"][n],latitude+","+longitude])

datos_montes = pd.DataFrame(datos_montes,columns=['nome','coordenadas'])

In [ ]:
datos_montes

In [76]:
#Veo si está todo bien, es curioso que todas las montañas más altas del mundo se encuentren en la misma zona

In [ ]:
m = folium.Map(location=[43, -8.20],zoom_start=2)

folium.GeoJson(df_mundo,name="paises", style_function=lambda x:style1).add_to(m)#aplico los shp
folium.GeoJson(df_rios,name="rios", style_function=lambda x:style2).add_to(m)


marker_cluster = MarkerCluster().add_to(m)
for n in range (0,46):#aplico los datos obtenidos en un bucle
  lat=datos_montes.coordenadas[n].split(",")[0]
  long=datos_montes["coordenadas"][n].split(",")[1]
  lat=float(lat)
  long=float(long)
  folium.Marker([lat,long],popup=(f'{datos_montes.nome[n]}'),icon=folium.Icon(color='gray',icon="glyphicon-pushpin")).add_to(marker_cluster)#añado las marcas al clúster
    
folium.LayerControl().add_to(m)
m

In [78]:
# Cargo el shp de todos los ríos de España
#https://www.mapa.gob.es/ide/metadatos/srv/spa/metadata.show?uuid=8e2570cd-7d55-46f0-877d-e69ff251abc8

In [ ]:
df_rios_es = gpd.read_file('//content/drive/MyDrive/COLAB/BDA/Rios A3_tcm30-176007/A3RIOS.shp')
df_rios_es.head(5)

In [80]:
len(df_rios_es)

359

In [81]:
#Ordeno los ríos según su longitud

In [82]:
by_leng = df_rios_es.sort_values('Shape_Leng')

In [83]:
#Me quedo con los 20 más largos

In [84]:
top_rios_es=by_leng[-20:]

In [ ]:
m = folium.Map(location=[43, -8.20],zoom_start=2)

folium.GeoJson(df_mundo,name="paises", style_function=lambda x:style1).add_to(m)
folium.GeoJson(df_rios,name="rios", style_function=lambda x:style2).add_to(m)
folium.GeoJson(top_rios_es).add_to(m)

marker_cluster = MarkerCluster().add_to(m)
for n in range (0,46):
  lat=datos_montes.coordenadas[n].split(",")[0]
  long=datos_montes["coordenadas"][n].split(",")[1]
  lat=float(lat)
  long=float(long)
  folium.Marker([lat,long],popup=(f'{datos_montes.nome[n]}'),icon=folium.Icon(color='gray',icon="glyphicon-pushpin")).add_to(marker_cluster)
    
folium.LayerControl().add_to(m)
m

In [86]:
# AMPLIACIÓN
#Mediante esta función gurado los puntos que obtengo del .gpx para posteriormente usarlos en el mapa

In [87]:
def overlayGPX(gpxData):
    gpx_file = open(gpxData, 'r')
    gpx = gpxpy.parse(gpx_file)
    points = []
    for track in gpx.tracks:
        for segment in track.segments:        
            for point in segment.points:
                points.append(tuple([point.latitude, point.longitude]))
    return (points)

In [88]:
#Cargo el gpx y paso la función
#https://www.rayyrosa.com/tracks

In [89]:
filePath = os.path.join("/content/drive/MyDrive/COLAB/BDA/ROMA-SANTIAGO-RAYYROSA.gpx")
puntos=overlayGPX(filePath)

In [90]:
#Resultado final

In [ ]:
m = folium.Map(location=[43, -8.20],zoom_start=2)

folium.TileLayer('Stamen Terrain').add_to(m)
folium.TileLayer('CartoDB Positron').add_to(m)

folium.GeoJson(df_mundo,name="paises", style_function=lambda x:style1).add_to(m)
folium.GeoJson(df_rios,name="rios", style_function=lambda x:style2).add_to(m)
folium.GeoJson(top_rios_es,name="rios_es").add_to(m)
folium.PolyLine(puntos,name="Camino de Santiago",color="red").add_to(m)#paso los puntos obtenidos

marker_cluster = MarkerCluster().add_to(m)
for n in range (0,46):
  lat=datos_montes.coordenadas[n].split(",")[0]
  long=datos_montes["coordenadas"][n].split(",")[1]
  lat=float(lat)
  long=float(long)
  folium.Marker([lat,long],popup=(f'{datos_montes.nome[n]}'),icon=folium.Icon(color='gray',icon="glyphicon-pushpin")).add_to(marker_cluster)
    
folium.LayerControl().add_to(m)
m

In [93]:
m.save('/content/drive/MyDrive/COLAB/BDA/mapa_mihai.html')